In [1]:
# For testing the performance of Tensor Ring Stochastic Gradient Descent (TR-SGD) on Recommendation System
from tr_functions import *
import timeit
from datetime import datetime
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
tags = pd.read_csv(r'C:\Users\LHY\Dropbox\Gunosy\Jupyter\data\ml-hetrec/user_taggedmovies-timestamps.dat', sep=None,  engine='python')
# tags = pd.read_csv(r'C:\Users\LHY\Dropbox\Gunosy\Jupyter\data\ml-25m/tags.csv', engine='python')
tags = tags.sort_values("timestamp", inplace=False, ascending=True)

def ml_tag_info(tags):
    user_count = tags.iloc[:,0].value_counts()
    movie_count = tags.iloc[:,1].value_counts()
    tags_count = tags.iloc[:,2].value_counts()
    print('Data info:')
    print('number of tagging is', tags.shape[0])
    print('number of user is', user_count.shape[0])
    print('number of movie is', movie_count.shape[0])
    print('number of different tags is', tags_count.shape[0])
    print('tags is from', datetime.fromtimestamp(tags.iloc[:,3].min()/1000), 'to', datetime.fromtimestamp(tags.iloc[:,3].max()/1000))
    print('time gap is', (tags.iloc[:,3].max() - tags.iloc[:,3].min())/365/24/3600/1000,'years')
#print('data sparsity is', tags_count.shape[0]tags.iloc[:,0].value_counts().shape[0])
ml_tag_info(tags)
tags[0:3]

Data info:
number of tagging is 47957
number of user is 2113
number of movie is 5908
number of different tags is 9079
tags is from 2005-12-24 03:13:00 to 2009-01-05 07:37:13
time gap is 3.036119133688483 years


,userID,movieID,tagID,timestamp
47575,71331,1396,542,1135361580000
11358,22198,4011,520,1135429431000
11355,22198,2329,8336,1135429508000


In [3]:
# filtering tags
tags_select = tags.copy()
num = 3

tags_count = tags_select.iloc[:,2].value_counts()
for i in range(tags_count.shape[0]):
    if np.array(tags_count)[i] < num:
        tags_select = tags_select[~tags_select['tagID'].isin([tags_count.index[i]])]
        
tags_count = tags_select.iloc[:,2].value_counts()
ml_tag_info(tags_select)
print(tags_count[tags_count.shape[0]-3:])

Data info:
number of tagging is 40000
number of user is 1777
number of movie is 5554
number of different tags is 2561
tags is from 2005-12-24 22:03:51 to 2009-01-05 07:06:38
time gap is 3.0339094051243025 years
5241    3
2185    3
2047    3
Name: tagID, dtype: int64


In [4]:
# filtering users
user_count = tags_select.iloc[:,0].value_counts()
for i in range(user_count.shape[0]):
    if np.array(user_count)[i] < num:
        tags_select = tags_select[~tags_select['userID'].isin([user_count.index[i]])]
        
user_count = tags_select.iloc[:,0].value_counts()
print(user_count[user_count.shape[0]-3:])
ml_tag_info(tags_select)

19820    3
66929    3
26589    3
Name: userID, dtype: int64
Data info:
number of tagging is 38816
number of user is 829
number of movie is 5499
number of different tags is 2556
tags is from 2005-12-24 22:03:51 to 2009-01-05 07:06:38
time gap is 3.0339094051243025 years


In [5]:
# filtering movies
movie_count = tags_select.iloc[:,1].value_counts()
for i in range(movie_count.shape[0]):
    if np.array(movie_count)[i] < num:
        tags_select = tags_select[~tags_select['movieID'].isin([movie_count.index[i]])]
        
movie_count = tags_select.iloc[:,1].value_counts()
print(movie_count[movie_count.shape[0]-3:])
ml_tag_info(tags_select)

58975    3
3261     3
26398    3
Name: movieID, dtype: int64
Data info:
number of tagging is 35553
number of user is 828
number of movie is 3125
number of different tags is 2539
tags is from 2005-12-24 22:03:51 to 2009-01-05 07:06:38
time gap is 3.0339094051243025 years


In [6]:
tags_select.userID = tags_select.userID.astype('category')
tags_select.movieID = tags_select.movieID.astype('category')
tags_select.timestamp = tags_select.timestamp.astype('category')
tags_select.tagID = tags_select.tagID.astype('category')
tags_select['userIndex'] = tags_select.userID.cat.codes
tags_select['movieIndex'] = tags_select.movieID.cat.codes
tags_select['tagIndex'] = tags_select.tagID.cat.codes
tags_select['timeIndex'] = tags_select.timestamp.cat.codes

tags_select = tags_select.sort_values("tagIndex", inplace=False, ascending=True)
tags_select[0:3]

,userID,movieID,tagID,timestamp,userIndex,movieIndex,tagIndex,timeIndex
35412,48717,45442,2,1161647764000,562,2820,0,10581
38294,55865,31367,2,1201473648000,635,2590,0,27371
13718,23172,3697,2,1205721076000,288,1329,0,28150


In [7]:
seq = tags_select['timeIndex'].max()//36
time_seg = []
for i in range(tags_select.shape[0]):
    if tags_select.iloc[i,7]//seq != 36:
        time_seg.append(tags_select.iloc[i,7]//seq)
    else:
        time_seg.append(35)
tags_select.insert(8, 'timeSeg', time_seg)
tags_select[0:3]

,userID,movieID,tagID,timestamp,userIndex,movieIndex,tagIndex,timeIndex,timeSeg
35412,48717,45442,2,1161647764000,562,2820,0,10581,11
38294,55865,31367,2,1201473648000,635,2590,0,27371,30
13718,23172,3697,2,1205721076000,288,1329,0,28150,31


In [8]:
tags_select.to_csv(r'C:\Users\LHY\Dropbox\Gunosy\Jupyter\data\hetrec_3core_implicit.csv') 

In [9]:
X_train, X_test, _, _ = train_test_split(tags_select[['userIndex', 'movieIndex', 'tagIndex', 'timeSeg']].values, 
                                                    tags_select['userIndex'].values, test_size=0.2, random_state=0)

y_train = np.ones(X_train.shape[0])
y_test = np.ones(X_test.shape[0])

print(X_train.shape, X_test.shape,y_train.shape, y_test.shape)

(28442, 4) (7111, 4) (28442,) (7111,)


In [10]:
# Train on the training data

# TR-SGD

tensor_size = [tags_select['userIndex'].max()  + 1,tags_select['movieIndex'].max() + 1, tags_select['tagIndex'].max()+1, 36]
print('tensor size is', tensor_size)

tr_rank = [10, 10, 10, 10]

start1 = timeit.default_timer()
cores = TR_SGD(X_train, y_train, tensor_size, tr_rank, learning_rate = 0.01, epoch = 10, alpha = 0.01, beta = 1)
stop1 = timeit.default_timer()
print('spend time is', stop1 - start1)

y_approx = []
for i in range(X_train.shape[0]):
    y_approx.append(np.trace(cores_multi(cores, np.array(X_train[i]), remove_n = None)))
MAE = evaluation_MAE(y_train, y_approx)
print('order-4 MAE on training set is', MAE)

y_test_approx = []
for i in range(X_test.shape[0]):
    y_test_approx.append(np.trace(cores_multi(cores, np.array(X_test[i]), remove_n = None)))
MAE = evaluation_MAE(y_test, y_test_approx)
print('order-4 MAE on test set is', MAE)


tensor size is [828, 3125, 2539, 36]
epoch 1 finish. RSE of random 100 samples is 0.7573726208874021
epoch 2 finish. RSE of random 100 samples is 0.3973248503143981
epoch 3 finish. RSE of random 100 samples is 0.3693706142941528
epoch 4 finish. RSE of random 100 samples is 0.24639376180497902
epoch 5 finish. RSE of random 100 samples is 0.19608960705507664
epoch 6 finish. RSE of random 100 samples is 0.13344983888540848
epoch 7 finish. RSE of random 100 samples is 0.14303677489210295
epoch 8 finish. RSE of random 100 samples is 0.10038444399139353
epoch 9 finish. RSE of random 100 samples is 0.14220826680718157
epoch 10 finish. RSE of random 100 samples is 0.12352811883605072
spend time is 24.0397383
order-4 MAE on training set is 0.09011067641512653
order-4 MAE on test set is 0.10960595862845589
